In [182]:
from time import time
from time import sleep
from datetime import datetime
from requests import get
from random import randint
from random import choice
from IPython.core.display import clear_output
from warnings import warn
import json
import csv

data = []

def get_all_jobs(pages):
    requests = 0
    start_time = time()
    total_runtime = datetime.now()
    user_agent_list = [
        # Chrome
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
        'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.71 Safari/537.36'
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
        'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'
        'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'
        'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'

        # Firefox
        'Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1'
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0'
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:18.0) Gecko/20100101 Firefox/18.0'
        'Mozilla/5.0 (X11; U; Linux Core i7-4980HQ; de; rv:32.0; compatible; JobboerseBot; http://www.jobboerse.com/bot.htm) Gecko/20100101 Firefox/38.0'
        'Mozilla/5.0 (Windows NT 5.1; rv:36.0) Gecko/20100101 Firefox/36.0'
        'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
        'Mozilla/5.0 (Windows NT 5.1; rv:33.0) Gecko/20100101 Firefox/33.0'
        'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0'
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
    ]

    for page in pages:
        try:
            user_agent = choice(user_agent_list)
            headers = {'User-Agent': user_agent}

            response = get('https://www.amazon.jobs/en/search.json?base_query=&city=&country=USA&county=&%27%27facets%5B%5D=location&facets%5B%5D=business_category&facets%5B%5D=category&%27%27facets%5B%5D=schedule_type_id&facets%5B%5D=employee_class&facets%5B%5D=normalized_location%27%27&facets%5B%5D=job_function_id&job_function_id%5B%5D=job_function_corporate_80rdb4&%27%27latitude=&loc_group_id=&loc_query=USA&longitude=&%27&offset={}&query_options=&%27%27radius=24km&region=&result_limit=10&business_category%5B%5D=operations-technology&business_category%5B%5D=transport&business_category%5B%5D=retail&schedule_type_id%5B%5D=Full-Time&%27%27sort=relevant'.format(page),
                           headers=headers,
                           # You will need your own Crawlera account and place below.
                           # proxies={"http": "http://xxxxxxxxxxxxxxxxxxxxxxx:@proxy.crawlera.com:8010/"}
                           )

            # Monitor the frequency of requests
            requests += 1

            # Pauses the loop between 8 - 15 seconds and marks the elapsed time
            sleep(randint(3, 6))
            current_time = time()
            elapsed_time = current_time - start_time
            print("Amazon Request:{}; Frequency: {} request/s; Total Run Time: {}".format(requests,
                  requests / elapsed_time, datetime.now() - total_runtime))
            clear_output(wait=True)

            # Throw a warning for non-200 status codes
            if response.status_code != 200:
                warn("Request: {}; Status code: {}".format(requests, response.status_code))

            # Set page requests. Break the loop if number of requests is greater than expected
            if requests > 999:
                warn("Number of requests was greater than expected.")
                break
            yield from get_job_infos(response)

        except AttributeError as e:
            print(e)
            continue
            
            
def get_job_infos(response):
    amazon_jobs = json.loads(response.text)
    for website in amazon_jobs['jobs']:
        JobId =website['id_icims']
        CompanyName = website['company_name']
        JobTitle = website['title']
        JobLocation = website['normalized_location']
        CountryCD = website['country_code']
        BusinessCategory = website['business_category']
        JobCategory = website['job_category']
        JobFamily = website['job_family']
        JobScheduleType = website['job_schedule_type']
        PostDate = website['posted_date']
        LastUpdate = website['updated_time']
        Team = website['team']['label']
        JobDescShort = website['description_short']
        JobDesc = website['description'].replace('Â','').replace('Â·','').replace('<br>','').replace('<br/>','').replace('â€™s','')
        JobQfBasic = website['basic_qualifications'].replace('Â','').replace('Â·','').replace('<br>','').replace('<br/>','').replace('â€™s','')
        JobQfPref = website['preferred_qualifications'].replace('Â','').replace('Â·','').replace('<br>','').replace('<br/>','').replace('â€™s','')
        job_link = 'https://www.amazon.jobs' + website['job_path']
        yield JobId, CompanyName, JobTitle, JobLocation, CountryCD, BusinessCategory, JobCategory, JobFamily,JobScheduleType, PostDate, LastUpdate, Team, JobDesc,JobQfBasic,JobQfPref

def main():
    # Page range starts from 0 and the middle value increases by 10 each page.
    # pages = [str(i) for i in range(0, 9990, 10)] 
    pages = [str(i) for i in range(0, 50, 10)]


    # Writes to csv file
    with open('amazon_jobs_NLP2.csv', "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["JobId", "CompanyName", "JobTitle", "JobLocation", "CountryCD", "BusinessCategory", "JobCategory", "JobFamily", "JobScheduleType", "PostDate", "LastUpdate", "Team", "JobDesc", "JobQfBasic", "JobQfPref"])
        writer.writerows(get_all_jobs(pages))

if __name__ == "__main__":
    main()




Amazon Request:5; Frequency: 0.2055775106073787 request/s; Total Run Time: 0:00:24.321726


In [184]:
import heapq
import pandas as pd

import pandas as pd
import re
import nltk
#nltk.download()
import heapq

stopwords = nltk.corpus.stopwords.words('english')
descsummarydata = []
columns = ["JobId", "CompanyName", "JobTitle", "JobLocation", "CountryCD", "BusinessCategory", "JobCategory", "JobFamily", "JobScheduleType", "PostDate", "LastUpdate", "Team", "JobDesc", "JobQfBasic", "JobQfPref"]
data = pd.read_csv('amazon_jobs_NLP2.csv')
#data = pd.read_csv('amazon_jobs_NLP2.csv', usecols=['JobDesc'])
#print(data)

for index in range(len(data)):
        sentence_list = nltk.sent_tokenize(data['JobDesc'].iloc[index] )
        word_frequencies = {}
        for word in nltk.word_tokenize(data['JobDesc'].iloc[index]):
                if word not in stopwords:
                    if word not in word_frequencies.keys():
                        word_frequencies[word] = 1
                    else:
                        word_frequencies[word] += 1

        sentence_scores = {}
        for sent in sentence_list:
                for word in nltk.word_tokenize(sent.lower()):
                    if word in word_frequencies.keys():
                        if len(sent.split(' ')) < 30:
                            if sent not in sentence_scores.keys():
                                sentence_scores[sent] = word_frequencies[word]
                            else:
                                sentence_scores[sent] += word_frequencies[word]

        summary_sentences = heapq.nlargest(3, sentence_scores, key=sentence_scores.get)
        summary = ' '.join(summary_sentences)
        descsummarydata.append([summary])
        #print(summary)
index=index+1

descsummaryvalues = pd.Series(descsummarydata)
#print(descsummaryvalues)
data.insert(loc=0, column='DescSummary', value=descsummaryvalues)
data['DescSummary']= data['DescSummary'].map(lambda x: str(x)[:-2]).map(lambda x: str(x)[2:])
data.to_csv('NLPtest.csv')

#data['result'] = data['result'].map(lambda x: x.lstrip('+-').rstrip('aAbBcC'))
#data['result'] = data['result'].map(lambda x: str(x)[:-1])

#website['basic_qualifications'].replace('Â','')
#.replace('[','')